In [ ]:
import scanpy as sc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import os
from scipy.stats import mannwhitneyu

In [ ]:
basepath = '/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/figures/'
dpi = 300
region_cmap = {str(i):sc.pl.palettes.default_102[i] for i in range(100)}

In [ ]:
adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples.h5ad')

In [ ]:
# cell types and regions
adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_ctyped_level2_region.h5ad')

# Cell typing

In [ ]:
sc.pp.pca(adata, n_comps=50)
sc.external.pp.harmony_integrate(adata, 'dataset')
sc.pp.neighbors(adata, use_rep='X_pca_harmony')
sc.tl.leiden(adata)
sc.tl.umap(adata)

In [ ]:
# sc.tl.leiden(adata, resolution=0.55)
sc.tl.leiden(adata, resolution=0.45)

In [ ]:
sc.set_figure_params(figsize=(7,5))
sc.pl.embedding(adata, basis='spatial', color='leiden', s=10)
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap', color='leiden', s=10)
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap', color='dataset', s=10)

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='leiden')

In [ ]:
# group = '0'
# subdata = adata[adata.obs['leiden']==group]
# sc.pl.rank_genes_groups_dotplot(subdata, groupby='leiden', n_genes=5)
sc.pl.rank_genes_groups_dotplot(adata, groupby='leiden', n_genes=5, dendrogram=False)

In [ ]:
leiden2ctype = {
    '0': 'Exc1',
    '1': 'Exc2',
    '2': 'Oligo',
    '3': 'Astro',
    '4': 'Exc3',
    '5': 'Peri',
    '6': 'Inh1',
    '7': 'Inh2',
    '8': 'Endo',
    '9': 'OPC',
    '10': 'Exc4',
}

adata.obs['celltype'] = adata.obs['leiden'].map(leiden2ctype).astype('category')

In [ ]:
# adata.write('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_ctyped.h5ad')
adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_ctyped.h5ad')

# Cell typing 2 (subclustering)

In [ ]:
basepath = '/stanley/WangLab/kamal/code/remote_notebooks/spin_supps/figures/'

## Level1

In [ ]:
sc.pp.pca(adata, n_comps=20)
sc.external.pp.harmony_integrate(adata, 'dataset', adjusted_basis='X_pca_harmony_level1')
sc.pp.neighbors(adata, use_rep='X_pca_harmony_level1')
sc.tl.umap(adata)
adata.obsm['X_umap_level1'] = adata.obsm['X_umap']
del adata.obsm['X_umap']

In [ ]:
sc.tl.leiden(adata, resolution=0.1, key_added='level1')

### temp

In [ ]:
sc.set_figure_params(figsize=(7,5))
sc.pl.embedding(adata, basis='spatial', color='level1', s=10, title='', legend_loc=None, return_fig=True, palette=sc.pl.palettes.default_102)
plt.axis('off')
figname = f'tissue_colored_by_level1.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap_level1', color='level1', s=8, title='',palette=sc.pl.palettes.default_102, legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_level1.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

### temp

In [ ]:
species_cmap = plt.cm.get_cmap('tab10').colors[:2][::-1]

In [ ]:
sc.set_figure_params(figsize=(5,5))
random_idxs = np.random.choice(np.arange(len(adata)), size=len(adata), replace=False)
sc.pl.embedding(adata[random_idxs], basis='X_umap_level1', color='dataset', s=8, title='', palette=species_cmap, legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_level1_colored_by_dataset.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
# Final as of 2023-06-29
adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_ctyped_level2_inh2.h5ad')

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='level1')

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, groupby='level1', n_genes=3, dendrogram=False, show=False)
figname = f'level1_markers.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
leiden2level1 = {
    '0': 'Excitatory',
    '1': 'Non-neuronal',
    '2': 'Inhibitory',
}

adata.obs['level1'] = adata.obs['level1'].map(leiden2level1).astype('category')

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, groupby='level1', n_genes=5, dendrogram=False, show=False)
figname = f'level1_markers.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

## Level2

### Excitatory

In [ ]:
subdata_exc = adata[adata.obs['level1']=='Excitatory']

In [ ]:
sc.pp.pca(subdata_exc, n_comps=20)
sc.external.pp.harmony_integrate(subdata_exc, 'dataset')
sc.pp.neighbors(subdata_exc, use_rep='X_pca_harmony')
sc.tl.umap(subdata_exc)
subdata_exc.obsm['X_umap_level2'] = subdata_exc.obsm['X_umap']
del subdata_exc.obsm['X_umap']

In [ ]:
sc.tl.leiden(subdata_exc, resolution=0.15, key_added='level2')

In [ ]:
sc.set_figure_params(figsize=(7,5))
sc.pl.embedding(subdata_exc, basis='spatial', color='level2', s=10, title='', legend_loc=None, return_fig=True, palette=sc.pl.palettes.default_102)
plt.axis('off')
figname = f'tissue_colored_by_level2_exc.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(subdata_exc, basis='X_umap_level2', color='level2', s=8, title='',palette=sc.pl.palettes.default_102, legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_level2_exc.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
species_cmap = plt.cm.get_cmap('tab10').colors[:2][::-1]

In [ ]:
sc.set_figure_params(figsize=(5,5))
random_idxs = np.random.choice(np.arange(len(subdata_exc)), size=len(subdata_exc), replace=False)
sc.pl.embedding(subdata_exc[random_idxs], basis='X_umap_level2', color='dataset', s=8, title='', palette=species_cmap, legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_level2_colored_by_dataset_exc.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
# Final as of 2023-06-29
adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_ctyped_level2_inh2.h5ad')
subdata_exc = adata[adata.obs['level1']=='Excitatory']
subdata_exc

In [ ]:
leiden2level2 = {
    'IT': 'IT',
    'ET': 'ET',
    'OB': 'AON',
}

subdata_exc.obs['level2'] = subdata_exc.obs['level2'].map(leiden2level2).astype('category')

In [ ]:
sc.tl.rank_genes_groups(subdata_exc, groupby='level2')

In [ ]:
sc.pl.rank_genes_groups_dotplot(subdata_exc, groupby='level2', n_genes=3, dendrogram=False, show=False)
figname = f'level2_markers_exc.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
leiden2level2 = {
    '0': 'IT',
    '1': 'ET',
    '2': 'AON',
}

subdata_exc.obs['level2'] = subdata_exc.obs['level2'].map(leiden2level2).astype('category')

In [ ]:
sc.pl.rank_genes_groups_dotplot(subdata_exc, groupby='level2', n_genes=5, dendrogram=False, show=False)
figname = f'level2_markers_exc.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

### Inhibitory

In [ ]:
adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_ctyped_level2.h5ad')

In [ ]:
subdata_inh = adata[adata.obs['level1']=='Inhibitory']

In [ ]:
# sc.pp.pca(subdata_inh, n_comps=20)
sc.pp.pca(subdata_inh, n_comps=10)
sc.external.pp.harmony_integrate(subdata_inh, 'dataset')
sc.pp.neighbors(subdata_inh, use_rep='X_pca_harmony')
sc.tl.umap(subdata_inh)
subdata_inh.obsm['X_umap_level2'] = subdata_inh.obsm['X_umap']
del subdata_inh.obsm['X_umap']

In [ ]:
# sc.tl.leiden(subdata_inh, resolution=0.06, key_added='level2')
sc.tl.leiden(subdata_inh, resolution=0.1, key_added='level2')

In [ ]:
sc.set_figure_params(figsize=(7,5))
sc.pl.embedding(subdata_inh, basis='spatial', color='level2', s=10, title='', legend_loc=None, return_fig=True, palette=sc.pl.palettes.default_102)
plt.axis('off')
figname = f'tissue_colored_by_level2_inh.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(subdata_inh, basis='X_umap_level2', color='level2', s=8, title='',palette=sc.pl.palettes.default_102, legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_level2_inh.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(subdata_inh, basis='X_umap_level2', color='level2', s=8, title='',palette=sc.pl.palettes.default_102, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_level2_inh.png'
savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
species_cmap = plt.cm.get_cmap('tab10').colors[:2][::-1]

In [ ]:
sc.set_figure_params(figsize=(5,5))
random_idxs = np.random.choice(np.arange(len(subdata_inh)), size=len(subdata_inh), replace=False)
sc.pl.embedding(subdata_inh[random_idxs], basis='X_umap_level2', color='dataset', s=8, title='', palette=species_cmap, legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_level2_colored_by_dataset_inh.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
# Final as of 2023-06-29
adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_ctyped_level2_inh2.h5ad')
subdata_inh = adata[adata.obs['level1']=='Inhibitory']
subdata_inh

In [ ]:
sc.tl.rank_genes_groups(subdata_inh, groupby='level2')

In [ ]:
sc.pl.rank_genes_groups_dotplot(subdata_inh, groupby='level2', n_genes=3, dendrogram=False, show=False)
figname = f'level2_markers_inh.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
# leiden2level2 = {
#     '0': 'Sst',
#     '1': 'Kit',
#     '2': 'Vip',
#     '3': 'Calb2',
#     '4': 'Pvalb',
# }

leiden2level2 = {
    '0': 'Pvalb',
    '1': 'Sst',
    '2': 'Cxcl14',
    '3': 'Vip',
}
subdata_inh.obs['level2'] = subdata_inh.obs['level2'].map(leiden2level2).astype('category')

In [ ]:
sc.pl.rank_genes_groups_dotplot(subdata_inh, groupby='level2', n_genes=5, dendrogram=False, show=False)
figname = f'level2_markers_inh.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(subdata_inh, basis='X_umap_level2', color='level2', s=8, title='',palette=sc.pl.palettes.default_102, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_level2_inh.png'
savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(7,5))
sc.pl.embedding(subdata_inh, basis='spatial', color='level2', s=10, title='', return_fig=True, palette=sc.pl.palettes.default_102)
plt.axis('off')
figname = f'tissue_colored_by_level2_inh.png'
savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

#### Rename Kit to Lamp5

In [ ]:
import scanpy as sc
import os
import matplotlib.pyplot as plt
basepath = '/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/figures/'
dpi = 300

adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_ctyped_level2.h5ad')
subdata_inh = adata[adata.obs['level1']=='Inhibitory']

level22leiden = {
    'Sst':'0',
    'Kit':'1',
    'Vip':'2',
    'Calb2':'3',
    'Pvalb':'4',
}

leiden2level2 = {
    '0': 'Sst',
    '1': 'Lamp5',
    '2': 'Vip',
    '3': 'Calb2',
    '4': 'Pvalb',
}

subdata_inh.obs['level2'] = subdata_inh.obs['level2'].map(level22leiden)
subdata_inh.obs['level2'] = subdata_inh.obs['level2'].cat.reorder_categories([str(i) for i in range(5)])

sc.tl.rank_genes_groups(subdata_inh, groupby='level2')
subdata_inh.obs['level2'] = subdata_inh.obs['level2'].map(leiden2level2).astype('category')
sc.pl.rank_genes_groups_dotplot(subdata_inh, groupby='level2', n_genes=5, dendrogram=False, show=False)#, groups=[str(i) for i in range(5)])
figname = f'level2_markers_inh.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

### Non-neuronal

In [ ]:
subdata_nn = adata[adata.obs['level1']=='Non-neuronal']

In [ ]:
sc.pp.pca(subdata_nn, n_comps=5)
sc.external.pp.harmony_integrate(subdata_nn, 'dataset')
sc.pp.neighbors(subdata_nn, use_rep='X_pca_harmony')
sc.tl.umap(subdata_nn)
subdata_nn.obsm['X_umap_level2'] = subdata_nn.obsm['X_umap']
del subdata_nn.obsm['X_umap']

In [ ]:
sc.tl.leiden(subdata_nn, resolution=0.15, key_added='level2')

In [ ]:
sc.set_figure_params(figsize=(7,5))
sc.pl.embedding(subdata_nn, basis='spatial', color='level2', s=10, title='', legend_loc=None, return_fig=True, palette=sc.pl.palettes.default_102)
plt.axis('off')
figname = f'tissue_colored_by_level2_nn.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(subdata_nn, basis='X_umap_level2', color='level2', s=8, title='',palette=sc.pl.palettes.default_102, legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_level2_nn.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
species_cmap = plt.cm.get_cmap('tab10').colors[:2][::-1]

In [ ]:
sc.set_figure_params(figsize=(5,5))
random_idxs = np.random.choice(np.arange(len(subdata_nn)), size=len(subdata_nn), replace=False)
sc.pl.embedding(subdata_nn[random_idxs], basis='X_umap_level2', color='dataset', s=8, title='', palette=species_cmap, legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_level2_colored_by_dataset_nn.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
# Final as of 2023-06-29
adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_ctyped_level2_inh2.h5ad')
subdata_nn = adata[adata.obs['level1']=='Non-neuronal']
subdata_nn

In [ ]:
sc.tl.rank_genes_groups(subdata_nn, groupby='level2')

In [ ]:
sc.pl.rank_genes_groups_dotplot(subdata_nn, groupby='level2', n_genes=3, dendrogram=False, show=False)
figname = f'level2_markers_nn.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
leiden2level2 = {
    '0': 'Astro',
    '1': 'Oligo1',
    '2': 'Endo',
    '3': 'Oligo2',
    '4': 'Peri',
    '5': 'OPC',
}

subdata_nn.obs['level2'] = subdata_nn.obs['level2'].map(leiden2level2).astype('category')

In [ ]:
sc.pl.rank_genes_groups_dotplot(subdata_nn, groupby='level2', n_genes=5, dendrogram=False, show=False)
figname = f'level2_markers_nn.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(subdata_nn, basis='X_umap_level2', color='level2', s=8, title='',palette=sc.pl.palettes.default_102, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_level2_nn.png'
savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

## Combine subclustering

In [ ]:
# From first run
adata_level2 = sc.concat([subdata_exc, subdata_inh, subdata_nn])

In [ ]:
# From inh sub repeat above
subdata_noninh = adata[adata.obs['level1']!='Inhibitory']
adata_level2 = sc.concat((subdata_inh, subdata_noninh))

In [ ]:
adata_level2

In [ ]:
adata.obs['idx'] = list(range(len(adata)))
idx = adata.obs['idx'][adata_level2.obs.index]
# adata_level2.obs['level1'] = adata[idx].obs['level1']
# adata_level2.obsm['X_pca_harmony_level1'] = adata[idx].obsm['X_pca_harmony_level1']
# adata_level2.obsm['X_umap_level1'] = adata[idx].obsm['X_umap_level1']
adata_level2.obsm['X_pca_spin'] = adata[idx].obsm['X_pca_spin']
adata_level2.obsm['X_umap_spin'] = adata[idx].obsm['X_umap_spin']
adata_level2.obsm['nbr_idxs'] = adata[idx].obsm['nbr_idxs']
adata_level2.obsm['nbr_dists'] = adata[idx].obsm['nbr_dists']
adata_level2.varm['X_pca_spin'] = adata[idx].varm['X_pca_spin']
adata_level2.uns['n_nbrs'] = adata[idx].uns['num_nbrs']
adata_level2.uns['n_samples'] = adata[idx].uns['num_samples']
adata_level2.uns['region'] = adata[idx].uns['region']
adata_level2.layers['smooth'] = adata[idx].layers['smooth']
adata_level2.obsp['region_connectivities'] = adata[idx].obsp['region_connectivities']
adata_level2.obsp['region_distances'] = adata[idx].obsp['region_distances']

In [ ]:
adata_level2.write('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_ctyped_level2_inh2.h5ad')

In [ ]:
basepath = '/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/figures/'

In [ ]:
# Final as of 2023-06-29
adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_ctyped_level2_inh2.h5ad')

In [ ]:
ctypes = adata.obs['level2'].value_counts().index
ctype_cmap = {ctypes[i]:sc.pl.palettes.default_102[i] for i in range(len(ctypes))}

In [ ]:
sc.set_figure_params(figsize=(7,5))
sc.pl.embedding(adata, basis='spatial', color='level2', s=10, title='', legend_loc=None, return_fig=True, palette=ctype_cmap)
plt.axis('off')
figname = f'tissue_colored_by_level2_all.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap_level1', color='level2', s=8, title='',palette=ctype_cmap, legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_level2_all.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap_level1', color='level2', s=8, title='',palette=ctype_cmap, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_level2_all.png'
savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap_level1', color='level2', groups='Pvalb', s=8, title='',palette=ctype_cmap, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_level2_all.png'
savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
random_idxs = np.random.choice(np.arange(len(adata)), size=len(adata), replace=False)
sc.pl.embedding(adata[random_idxs], basis='X_umap_level1', color='dataset', s=8, title='', legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_level2_species.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

# Tissue and latent colored by cell type

In [ ]:
# Modified below to roughly match region coloring
# ctypes = adata.obs['celltype'].unique()
ctypes = adata.obs['level2'].unique()
ctype_cmap = {ctypes[i]:sc.pl.palettes.default_102[i] for i in range(len(ctypes))}

In [ ]:
sc.set_figure_params(figsize=(7,5))
sc.pl.embedding(adata, basis='spatial', color='level2', s=10, title='', legend_loc=None, return_fig=True, palette=ctype_cmap)
plt.axis('off')
figname = f'tissue_colored_by_celltype.png'
savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(7,5))
sc.pl.embedding(adata, basis='spatial', color='level2', s=10, title='', legend_loc=None, return_fig=True, palette=ctype_cmap)
plt.axis('off')
figname = f'tissue_colored_by_celltype.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap', color='level2', s=8, title='',palette=ctype_cmap, legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_celltype.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap', color='level2', s=8, legend_loc='on data', palette=ctype_cmap)

In [ ]:
sc.set_figure_params(figsize=(5,5))
random_idxs = np.random.choice(np.arange(len(adata)), size=len(adata), replace=False)
sc.pl.embedding(adata[random_idxs], basis='X_umap', color='dataset', s=5, title='', legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_species_ctype.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

### Assign colors

In [ ]:
# sc.set_figure_params(figsize=(5,5))
# for ctype in ctypes:
#     sc.pl.embedding(adata, basis='X_umap', color='celltype', groups=ctype, s=3, title='', return_fig=True)
#     plt.axis('off')

In [ ]:
ctypes = adata.obs['celltype'].unique()
ctype_cmap = {ctypes[i]:sc.pl.palettes.default_102[i] for i in range(len(ctypes))}

In [ ]:
print(list(ctypes))

In [ ]:
ctype_cmap[ctypes[4]] = ctype_cmap['L5 ET']
ctype_cmap['L5 ET'] = sc.pl.palettes.default_102[4]

In [ ]:
ctype_cmap[ctypes[6]] = ctype_cmap['L6 CT']
ctype_cmap['L6 CT'] = sc.pl.palettes.default_102[6]

In [ ]:
ctype_cmap[ctypes[1]] = ctype_cmap['Microglia/PVM']
ctype_cmap['Microglia/PVM'] = sc.pl.palettes.default_102[1]

In [ ]:
ctype_cmap[ctypes[9]] = ctype_cmap['Oligo']
ctype_cmap['Oligo'] = sc.pl.palettes.default_102[9]

In [ ]:
sc.set_figure_params(figsize=(3,3))
for i in range(10):
    sc.pl.embedding(adata, basis='X_umap_spin', color='region', groups=str(i), palette=region_cmap, s=3, title='', return_fig=True)
    plt.axis('off')

# Tissue and latent colored by region

In [ ]:
# res = 0.5
# sc.tl.leiden(adata, resolution=res, neighbors_key='region', key_added='region')

In [ ]:
sc.set_figure_params(figsize=(7,5))
sc.pl.embedding(adata, basis='spatial', color='region', palette=region_cmap, s=10, title='', legend_loc=None, return_fig=True)
plt.axis('off')
# figname = f'tissue_colored_by_region.png'
# savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap_spin', color='region', palette=region_cmap, s=8, title='', legend_loc=None, return_fig=True)
plt.axis('off')
# figname = f'latent_colored_by_region.png'
# savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
# Filter out regions with fewer than 500 cells
good_regions = [str(i) for i in range(9)]
adata = adata[np.isin(adata.obs['region'],good_regions)]

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap_spin', color='dataset', s=8, title='', legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_species.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
random_idxs = np.random.choice(np.arange(len(adata)), size=len(adata), replace=False)
sc.pl.embedding(adata[random_idxs], basis='X_umap_spin', color='dataset', s=8, title='', legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_species.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

# Recolor

In [ ]:
subdata = adata[adata.obs['dataset']=='marmoset']
sc.set_figure_params(figsize=(3,8))
sc.pl.embedding(subdata, basis='spatial', color='region', s=20, title='', palette=region_cmap)

In [ ]:
leiden2region = {
    '0': 'L5',
    '1': 'L2/3',
    '2': 'L4',
    '3': 'L6',
    '4': 'meninges/L1',
    '5': 'fiber tract',
    '6': 'PIR',
    '7': 'TT',
    '8': 'aco',
}
adata.obs['leiden'] = adata.obs['region'].copy()
adata.obs['region'] = adata.obs['leiden'].map(leiden2region)
adata.obs['region'] = adata.obs['region'].astype('category')

## Use prior cmap

In [ ]:
olddata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/spatial_autocorrelation/adata_all_techniques.h5ad')
old_regions = olddata.obs['region_smooth_subsampled'].unique()
palette = sc.pl.palettes.default_102
old_cmap = {old_regions[i]:palette[i] for i in range(len(old_regions))}

In [ ]:
old_regions = set(old_regions)
new_regions = set(adata.obs['region'].unique())
shared_regions = old_regions.intersection(new_regions)
all_regions = old_regions.union(new_regions)

new_cmap = dict()
for region in shared_regions:
    new_cmap[region] = old_cmap[region]

print(all_regions-shared_regions)

In [ ]:
new_cmap['aco'] = old_cmap['DG']
new_cmap['TT'] = old_cmap['BLA']

In [ ]:
# Verify uniqueness of colors
len(new_cmap.values()) == len(set(new_cmap.values()))

In [ ]:
region_cmap = new_cmap

In [ ]:
sc.set_figure_params(figsize=(7,5))
sc.pl.embedding(adata, basis='spatial', color='region', palette=region_cmap, s=10, title='', legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'tissue_colored_by_region.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap_spin', color='region', palette=region_cmap, s=8, title='', legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_region.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

# Tissue zooms and latent colored by DiffMap

In [ ]:
adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_ctyped_level2.h5ad')

In [ ]:
ctx_regions = ['L2/3', 'L4', 'L5', 'L6']
adata = adata[np.isin(adata.obs['region'], ctx_regions)]

In [ ]:
sc.tl.diffmap(adata, neighbors_key='region')
adata.obs['depth'] = -adata.obsm['X_diffmap'][:,1]

In [ ]:
def get_zoom(adata, x, y, width, height):
    zoomdata = adata[adata.obsm['spatial'][:,0]>x]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,0]<x+width]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,1]>y]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,1]<y+height]
    return zoomdata

In [ ]:
# Original
# zoomdata_marmoset = get_zoom(adata[adata.obs['dataset']=='marmoset'], -4400, 12500, 1000, 2400)
# zoomdata_mouse = get_zoom(adata[adata.obs['dataset']=='mouse'], 2600, 10600, 1500, 3600)

In [ ]:
1600*3

In [ ]:
# marmoset_box = (-4400, 10800, 1400, 4200)
marmoset_box = (-4400, 10200, 1600, 4800)
subdata = adata[adata.obs['dataset']=='marmoset']

plt.figure(figsize=(1.5,4))
plt.scatter(*subdata.obsm['spatial'].T, s=1)
rect = Rectangle(marmoset_box[:2], marmoset_box[2], marmoset_box[3], fill=False, edgecolor='k')
plt.gca().add_patch(rect)
plt.axis('off')
plt.show()

In [ ]:
1800 * 3

In [ ]:
# mouse_box = (2600, 8200, 2050, 6150)
mouse_box = (2600, 8900, 1800, 5400)
subdata = adata[adata.obs['dataset']=='mouse']

plt.figure(figsize=(4,4))
plt.scatter(*subdata.obsm['spatial'].T, s=1)
rect = Rectangle(mouse_box[:2], mouse_box[2], mouse_box[3], fill=False, edgecolor='k')
plt.gca().add_patch(rect)
plt.axis('off')
plt.show()

In [ ]:
zoomdata_marmoset = get_zoom(adata[adata.obs['dataset']=='marmoset'], *marmoset_box)
zoomdata_mouse = get_zoom(adata[adata.obs['dataset']=='mouse'], *mouse_box)

In [ ]:
vmax = adata.obs['depth'].max()
vmin = adata.obs['depth'].min()
sc.set_figure_params(figsize=(2,6))
species = ['marmoset', 'mouse']
zoomdatas = [zoomdata_marmoset, zoomdata_mouse]
for i in range(len(species)):
    sc.pl.embedding(zoomdatas[i], basis='spatial', color='depth', cmap='viridis', s=50, return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor=None, vmax=vmax, vmin=vmin)
    figname = f'zoom_tissue_{species[i]}_colored_by_depth.png'
    savepath = os.path.join(basepath, figname)
    # plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

In [ ]:
adata_all = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples.h5ad')
good_regions = [str(i) for i in range(9)]
adata_all = adata_all[np.isin(adata_all.obs['region'],good_regions)]

In [ ]:
sc.set_figure_params(figsize=(6,6))
plt.scatter(*adata_all.obsm['X_umap_spin'].T, color=[0.7]*3, s=0.5)
plt.scatter(*adata.obsm['X_umap_spin'].T, c=adata.obs['depth'], cmap='viridis', s=0.5)
plt.axis('off')
figname = f'latent_colored_by_depth.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
# colorbar only
fig = plt.figure()
cax = plt.imshow(np.array([[0,1]]), cmap='viridis')
plt.gca().set_visible(False)
cbar = fig.colorbar(cax, ticks=[0, 1])
cbar.ax.set_yticklabels(['', ''])
figname = 'cbar_color_by_expr.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath)
plt.show()

# Tissue zooms colored by region

In [ ]:
adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_ctyped_level2.h5ad')

In [ ]:
adata.obs['region'].value_counts()

In [ ]:
# Filter out regions with fewer than 100 cells
good_regions = [str(i) for i in range(9)]
adata = adata[np.isin(adata.obs['region'],good_regions)]

In [ ]:
leiden2region = {
    '0': 'L5',
    '1': 'L2/3',
    '2': 'L4',
    '3': 'L6',
    '4': 'meninges/L1',
    '5': 'fiber tract',
    '6': 'PIR',
    '7': 'TT',
    '8': 'aco',
}
adata.obs['leiden'] = adata.obs['region'].copy()
adata.obs['region'] = adata.obs['leiden'].map(leiden2region)
adata.obs['region'] = adata.obs['region'].astype('category')

In [ ]:
sc.pl.embedding(adata, basis='spatial', color='region')

In [ ]:
olddata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/spatial_autocorrelation/adata_all_techniques.h5ad')
old_regions = olddata.obs['region_smooth_subsampled'].unique()
palette = sc.pl.palettes.default_102
old_cmap = {old_regions[i]:palette[i] for i in range(len(old_regions))}

In [ ]:
old_regions = set(old_regions)
new_regions = set(adata.obs['region'].unique())
shared_regions = old_regions.intersection(new_regions)
all_regions = old_regions.union(new_regions)

new_cmap = dict()
for region in shared_regions:
    new_cmap[region] = old_cmap[region]

print(all_regions-shared_regions)

In [ ]:
new_cmap['aco'] = old_cmap['DG']
new_cmap['TT'] = old_cmap['BLA']

In [ ]:
# Verify uniqueness of colors
len(new_cmap.values()) == len(set(new_cmap.values()))

In [ ]:
region_cmap = new_cmap

In [ ]:
adata.write('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_ctyped_level2_region.h5ad')

In [ ]:
def get_zoom(adata, x, y, width, height):
    zoomdata = adata[adata.obsm['spatial'][:,0]>x]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,0]<x+width]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,1]>y]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,1]<y+height]
    return zoomdata

In [ ]:
# marmoset_box = (-4400, 10800, 1400, 4200)
marmoset_box = (-4400, 10200, 1600, 4800)
subdata = adata[adata.obs['dataset']=='marmoset']

plt.figure(figsize=(1.5,4))
plt.scatter(*subdata.obsm['spatial'].T, s=1)
rect = Rectangle(marmoset_box[:2], marmoset_box[2], marmoset_box[3], fill=False, edgecolor='k')
plt.gca().add_patch(rect)
plt.axis('off')
plt.show()

In [ ]:
# mouse_box = (2600, 8200, 2050, 6150)
mouse_box = (2600, 8900, 1800, 5400)
subdata = adata[adata.obs['dataset']=='mouse']

plt.figure(figsize=(4,4))
plt.scatter(*subdata.obsm['spatial'].T, s=1)
rect = Rectangle(mouse_box[:2], mouse_box[2], mouse_box[3], fill=False, edgecolor='k')
plt.gca().add_patch(rect)
plt.axis('off')
plt.show()

In [ ]:
zoomdata_marmoset = get_zoom(adata[adata.obs['dataset']=='marmoset'], *marmoset_box)
zoomdata_mouse = get_zoom(adata[adata.obs['dataset']=='mouse'], *mouse_box)

In [ ]:
ctypes = adata.obs['level2'].value_counts().index
ctype_cmap = {ctypes[i]:sc.pl.palettes.default_102[i] for i in range(len(ctypes))}

In [ ]:
sc.set_figure_params(figsize=(2,6))
species = ['marmoset', 'mouse']
zoomdatas = [zoomdata_marmoset, zoomdata_mouse]
for i in range(len(species)):
    sc.pl.embedding(zoomdatas[i], basis='spatial', color='region', palette=region_cmap, s=50, return_fig=True, frameon=False, title='', legend_loc=None, edgecolor='k', linewidth=0.05)
    figname = f'zoom_tissue_{species[i]}_colored_by_regions.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

# Tissue zooms colored by ctype

In [ ]:
adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_ctyped_level2.h5ad')

In [ ]:
def get_zoom(adata, x, y, width, height):
    zoomdata = adata[adata.obsm['spatial'][:,0]>x]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,0]<x+width]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,1]>y]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,1]<y+height]
    return zoomdata

In [ ]:
# marmoset_box = (-4400, 10800, 1400, 4200)
marmoset_box = (-4400, 10200, 1600, 4800)
subdata = adata[adata.obs['dataset']=='marmoset']

plt.figure(figsize=(1.5,4))
plt.scatter(*subdata.obsm['spatial'].T, s=1)
rect = Rectangle(marmoset_box[:2], marmoset_box[2], marmoset_box[3], fill=False, edgecolor='k')
plt.gca().add_patch(rect)
plt.axis('off')
plt.show()

In [ ]:
# mouse_box = (2600, 8200, 2050, 6150)
mouse_box = (2600, 8900, 1800, 5400)
subdata = adata[adata.obs['dataset']=='mouse']

plt.figure(figsize=(4,4))
plt.scatter(*subdata.obsm['spatial'].T, s=1)
rect = Rectangle(mouse_box[:2], mouse_box[2], mouse_box[3], fill=False, edgecolor='k')
plt.gca().add_patch(rect)
plt.axis('off')
plt.show()

In [ ]:
zoomdata_marmoset = get_zoom(adata[adata.obs['dataset']=='marmoset'], *marmoset_box)
zoomdata_mouse = get_zoom(adata[adata.obs['dataset']=='mouse'], *mouse_box)

In [ ]:
ctypes = adata.obs['level2'].value_counts().index
ctype_cmap = {ctypes[i]:sc.pl.palettes.default_102[i] for i in range(len(ctypes))}

In [ ]:
sc.set_figure_params(figsize=(2,6))
species = ['marmoset', 'mouse']
zoomdatas = [zoomdata_marmoset, zoomdata_mouse]
for i in range(len(species)):
    sc.pl.embedding(zoomdatas[i], basis='spatial', color='level2', palette=ctype_cmap, s=50, return_fig=True, frameon=False, title='', legend_loc=None, edgecolor='k', linewidth=0.05)
    figname = f'zoom_tissue_{species[i]}_colored_by_ctype.png'
    savepath = os.path.join(basepath, figname)
    # plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

# Shared and species-specific region markers

In [ ]:
region = 'L2/3'

In [ ]:
# Shared
sc.tl.rank_genes_groups(adata, groupby='region')#, layer='smooth')
marker_df = sc.get.rank_genes_groups_df(adata, group=region)
shared_marker = marker_df['names'][0]

In [ ]:
plt.figure(figsize=(5,2))
plt.hist(marker_df['scores'], bins=30, color=sc.pl.palettes.default_102[0], linewidth=1, edgecolor='k')
# plt.hist(marker_df['scores'], bins=30, color=region_cmap['L2/3'], linewidth=1, edgecolor='k')
plt.grid(None)
plt.axis('off')
figname = f'region_deg_score_hist.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
plt.figure(figsize=(5,2))
plt.hist(marker_df['scores'], bins=30, color=sc.pl.palettes.default_102[0], linewidth=1, edgecolor='k')
# plt.hist(marker_df['scores'], bins=30, color=region_cmap['L2/3'], linewidth=1, edgecolor='k')
plt.grid(None)
plt.yticks([30],[30])
# plt.axis('off')
# figname = f'region_deg_score_hist.png'
# savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
marker_df['scores']

In [ ]:
marker_df['names']

In [ ]:
# Unique
xdata = adata[adata.obs['region']==region]
sc.tl.rank_genes_groups(xdata, groupby='dataset')#, layer='smooth')
marker_df = sc.get.rank_genes_groups_df(xdata, group='mouse')
mouse_markers = marker_df['names']
mouse_marker = mouse_markers[0]

In [ ]:
marker_df['scores']

In [ ]:
marker_df['names']

In [ ]:
plt.figure(figsize=(5,2))
plt.hist(marker_df['scores'], bins=30, color=sc.pl.palettes.default_102[0], linewidth=1, edgecolor='k')
# plt.hist(marker_df['scores'], bins=30, color=region_cmap['L2/3'], linewidth=1, edgecolor='k')
plt.grid(None)
plt.axis('off')
figname = f'species_L23_deg_score_hist.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
plt.figure(figsize=(5,2))
plt.hist(marker_df['scores'], bins=30, color=sc.pl.palettes.default_102[0], linewidth=1, edgecolor='k')
# plt.hist(marker_df['scores'], bins=30, color=region_cmap['L2/3'], linewidth=1, edgecolor='k')
plt.grid(None)
# plt.yticks([30],[30])
# plt.axis('off')
# figname = f'region_deg_score_hist.png'
# savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
# Unique
xdata = adata[adata.obs['region']==region]
sc.tl.rank_genes_groups(xdata, groupby='dataset')#, layer='smooth')
marker_df = sc.get.rank_genes_groups_df(xdata, group='marmoset')
marmoset_markers = marker_df['names']
marmoset_marker = marmoset_markers[0]

In [ ]:
marker_df['scores']

In [ ]:
marker_df['names']

In [ ]:
print(shared_marker, mouse_marker, marmoset_marker)

In [ ]:
X = sc.concat((zoomdata_marmoset, zoomdata_mouse))[:,[shared_marker]].X
vmax = X.max()
vmin = X.min()
cmaps = ['Blues', 'Oranges']

In [ ]:
sc.set_figure_params(figsize=(2,6))
species = ['marmoset', 'mouse']
zoomdatas = [zoomdata_marmoset, zoomdata_mouse]
for i in range(len(species)):
    sc.pl.embedding(zoomdatas[i], basis='spatial', color=shared_marker, s=120, cmap=cmaps[i],
    return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor='k', linewidth=0.05,
    )# vmin=vmin, vmax=vmax)
    # sc.pl.embedding(zoomdatas[i], basis='spatial', color=shared_marker, s=100, cmap='inferno_r', return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor=None)
    figname = f'zoom_tissue_{species[i]}_colored_by_{shared_marker}.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

In [ ]:
X = sc.concat((zoomdata_marmoset, zoomdata_mouse))[:,mouse_marker].X
vmax = X.max()
vmin = X.min()
cmaps = ['Blues', 'Oranges']

In [ ]:
sc.set_figure_params(figsize=(2,6))
species = ['marmoset', 'mouse']
zoomdatas = [zoomdata_marmoset, zoomdata_mouse]
for i in range(len(species)):
    sc.pl.embedding(zoomdatas[i], basis='spatial', color=mouse_marker, s=120, cmap=cmaps[i],
    return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor='k', linewidth=0.05,
    )#vmin=vmin, vmax=vmax)
    # sc.pl.embedding(zoomdatas[i], basis='spatial', color=mouse_marker, s=100, cmap='inferno_r', return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor=None)
    figname = f'zoom_tissue_{species[i]}_colored_by_{mouse_marker}.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

In [ ]:
sc.set_figure_params(figsize=(2,6))
species = ['marmoset', 'mouse']
zoomdatas = [zoomdata_marmoset, zoomdata_mouse]
for i in range(len(species)):
    sc.pl.embedding(zoomdatas[i], basis='spatial', color=marmoset_marker, s=120, cmap=cmaps[i],
    return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor='k', linewidth=0.05,
    )#vmin=vmin, vmax=vmax)
    # sc.pl.embedding(zoomdatas[i], basis='spatial', color=marmoset_marker, s=100, cmap='inferno_r', return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor=None)
    figname = f'zoom_tissue_{species[i]}_colored_by_{marmoset_marker}.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

## Full tissue plots of markers

In [ ]:
markers = [shared_marker, mouse_marker, marmoset_marker]
sc.set_figure_params(figsize=(7,5))
for marker in markers:
    for i in range(len(species)):
        sc.pl.embedding(adata, basis='spatial', color=marker, s=10, title='', colorbar_loc=None, return_fig=True, cmap=cmaps[i])
        plt.axis('off')
        figname = f'tissue_colored_by_{marker}_cmap{i}.png'
        savepath = os.path.join(basepath, figname)
        plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
        plt.show()

In [ ]:
sc.set_figure_params(figsize=(2,6))
species = ['marmoset', 'mouse']
zoomdatas = [zoomdata_marmoset, zoomdata_mouse]
for i in range(len(species)):
    sc.pl.embedding(zoomdatas[i], basis='spatial', color=marmoset_marker, s=120, cmap=cmaps[i],
    return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor='k', linewidth=0.05,
    )#vmin=vmin, vmax=vmax)
    # sc.pl.embedding(zoomdatas[i], basis='spatial', color=marmoset_marker, s=100, cmap='inferno_r', return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor=None)
    figname = f'zoom_tissue_{species[i]}_colored_by_{marmoset_marker}.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

# Continuous depth vs gene

In [ ]:
adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_ctyped_level2_inh2.h5ad')

In [ ]:
leiden2region = {
    '0': 'L5',
    '1': 'L2/3',
    '2': 'L4',
    '3': 'L6',
    '4': 'meninges/L1',
    '5': 'fiber tract',
    '6': 'PIR',
    '7': 'TT',
    '8': 'aco',
}
adata.obs['region'] = adata.obs['region'].map(leiden2region)
adata.obs['region'] = adata.obs['region'].astype('category')

In [ ]:
sc.set_figure_params(figsize=(7,5))
sc.pl.embedding(adata, basis='spatial', color='region', palette=sc.pl.palettes.default_102, s=10, title='', legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'tissue_colored_by_region.png'
savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap_spin', color='region', palette=sc.pl.palettes.default_102, s=8, title='', legend_loc=None, return_fig=True)
plt.axis('off')
figname = f'latent_colored_by_region.png'
savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)

In [ ]:
ctx_regions = ['L2/3', 'L4', 'L5', 'L6']
adata = adata[np.isin(adata.obs['region'], ctx_regions)]

In [ ]:
adata

In [ ]:
sc.pp.neighbors(adata, use_rep='X_pca_spin', key_added='region')
sc.tl.diffmap(adata, neighbors_key='region')
adata.obs['depth'] = -adata.obsm['X_diffmap'][:,1]

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.embedding(adata, basis='X_umap_spin', color='depth', s=8, title='', legend_loc=None, return_fig=True)
plt.axis('off')
plt.show()

In [ ]:
adata.write('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_ctyped_level2_inh2_depth_ctx.h5ad')

In [ ]:
# Make bins
num_bins = 50
max_depth = adata.obs['depth'].max()
min_depth = adata.obs['depth'].min()
increments = (max_depth-min_depth)/num_bins
bins = np.arange(min_depth, max_depth, increments)
adata.obs['depth_bin'] = np.digitize(adata.obs['depth'], bins=bins) - 1 # account for idxs

# Make expression nonnegative
adata.X -= adata.X.min()
adata.layers['smooth'] -= adata.layers['smooth'].min()

# Separate species
adata_marmoset = adata[adata.obs['dataset']=='marmoset']
adata_mouse = adata[adata.obs['dataset']=='mouse']
adatas = [adata_marmoset, adata_mouse]

In [ ]:
def get_hists(adatas, gene, start_bin=None, end_bin=None):

    hists = []
    for subdata in adatas:

        # Initialize a few vars
        hist = np.zeros(num_bins)
        depth = subdata.obs['depth_bin'].values
        expr = subdata[:,gene].X.flatten()
        # expr = subdata[:,gene].layers['smooth'].flatten()

        # Wanted to just use np.histogram here to get counts, but it has known binning diffs from np.digitize
        bin_counts = np.zeros(num_bins)
        nonzero_counts = subdata.obs['depth_bin'].value_counts().sort_index()
        for i in nonzero_counts.index:
            bin_counts[i] = nonzero_counts.loc[i]

        # Add each cell's expression into corresponding cell depth bin
        for i in range(len(subdata)):
            hist[depth[i]] += expr[i] / bin_counts[depth[i]]
        
        # Omit highest and lowest depths due to noise
        if not start_bin and end_bin:
            start_bin = num_bins // 10 # omit bottom 10% of depth
            end_bin = num_bins // 10 * 9 # omit top 10% of depth
        hist = hist[start_bin:end_bin]

        # Normalize wrt given dataset
        hist /= hist.sum()
        hists.append(hist)
    
    return hists

In [ ]:
start_bin = 5
end_bin = 45

In [ ]:
adata[:,'LAMP5'].X.flatten()

In [ ]:
species = ['marmoset', 'mouse']

for gene in ['LAMP5', 'CCK', 'SLC13A3', 'PLCH1']:
    hists = get_hists(adatas, gene, start_bin=start_bin, end_bin=end_bin)
    plt.figure(figsize=(10,3))
    for i in range(len(hists)):
        plt.plot(hists[i], label=species[i], linewidth=5)
    plt.axis('off')
    plt.ylim(0.015, 0.035)
    figname = f'depth_curve_{gene}.png'
    savepath = os.path.join(basepath, figname)
    # plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

In [ ]:
species = ['marmoset', 'mouse']

for gene in ['LAMP5', 'CCK']:
    plt.figure(figsize=(10,3))
    plt.plot(adata[:,gene].varm['marmoset_depth_dist'].flatten(), linewidth=5)
    plt.plot(adata[:,gene].varm['mouse_depth_dist'].flatten(), linewidth=5)
    plt.axis('off')
    # figname = f'depth_curve_{gene}.png'
    # savepath = os.path.join(basepath, figname)
    # plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

In [ ]:
num_top_genes = 5
most_similar_genes = adata.var['species_expr_diffs'].sort_values()[:num_top_genes].index
most_different_genes = adata.var['species_expr_diffs'].sort_values()[-num_top_genes:][::-1].index

In [ ]:
species = ['marmoset', 'mouse']

for gene in most_similar_genes:
    print(gene)
    plt.figure(figsize=(10,3))
    plt.plot(adata[:,gene].varm['marmoset_depth_dist'].flatten(), linewidth=5)
    plt.plot(adata[:,gene].varm['mouse_depth_dist'].flatten(), linewidth=5)
    plt.axis('off')
    figname = f'depth_curve_{gene}.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

In [ ]:
species = ['marmoset', 'mouse']

for gene in most_different_genes:
    plt.figure(figsize=(10,3))
    plt.title(gene)
    plt.plot(adata[:,gene].varm['marmoset_depth_dist'].flatten(), linewidth=5)
    plt.plot(adata[:,gene].varm['mouse_depth_dist'].flatten(), linewidth=5)
    plt.axis('off')
    # figname = f'depth_curve_{gene}.png'
    # savepath = os.path.join(basepath, figname)
    # plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

In [ ]:
gene = most_different_genes[1]
print(gene)
X = sc.concat((zoomdata_marmoset, zoomdata_mouse))[:,gene].X
vmax = X.max()
vmin = X.min()
cmaps = ['Blues', 'Oranges']

sc.set_figure_params(figsize=(2,6))
species = ['marmoset', 'mouse']
zoomdatas = [zoomdata_marmoset, zoomdata_mouse]
for i in range(len(species)):
    sc.pl.embedding(zoomdatas[i], basis='spatial', color=gene, s=100, cmap=cmaps[i], return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor=None)
    figname = f'zoom_tissue_{species[i]}_colored_by_{gene}.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

In [ ]:
gene = most_similar_genes[0]
print(gene)
X = sc.concat((zoomdata_marmoset, zoomdata_mouse))[:,gene].X
vmax = X.max()
vmin = X.min()
cmaps = ['Blues', 'Oranges']

sc.set_figure_params(figsize=(2,6))
species = ['marmoset', 'mouse']
zoomdatas = [zoomdata_marmoset, zoomdata_mouse]
for i in range(len(species)):
    sc.pl.embedding(zoomdatas[i], basis='spatial', color=gene, s=100, cmap=cmaps[i], return_fig=True, frameon=False, title='', colorbar_loc=None, edgecolor=None)
    figname = f'zoom_tissue_{species[i]}_colored_by_{gene}.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

No ISH data from japanese marmoset site, but evidence in macaques in the following preprint:

https://www.biorxiv.org/content/10.1101/2020.05.14.087601v2

Extended Data Fig. 4

In [ ]:
# colorbar only
for i in range(2):
    fig = plt.figure()
    cax = plt.imshow(np.array([[0,1]]), cmap=cmaps[i])
    plt.gca().set_visible(False)
    cbar = fig.colorbar(cax, ticks=[0, 1])
    cbar.ax.set_yticklabels(['', ''])
    figname = f'cbar_colored_by_{cmaps[i]}.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath)
    plt.show()

In [ ]:
plt.figure(figsize=(6,2))
plt.hist(adata.var['species_expr_diffs'], bins=30)
plt.xlabel('Species Difference')
plt.ylabel('Count')
plt.axis('off')
figname = f'gene_depth_diffs_hist.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
plt.figure(figsize=(6,2))
plt.hist(adata.var['species_expr_diffs'], bins=30)
plt.xlabel('Species Difference')
plt.ylabel('Count')
plt.show()

In [ ]:
adata.var['species_expr_diffs']['LAMP5']

In [ ]:
# adata.write('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_processed.h5ad')

# Cell type vs depth

In [ ]:
adata = sc.read_h5ad('/stanley/WangLab/kamal/code/remote_notebooks/mpfc_xspecies/marmoset_mouse_mpfc_spin_30nbrs_12samples_ctyped_level2_inh2_depth_ctx.h5ad')

# Don't include OB since looking at cortex
ctype_key = 'level2'
adata = adata[adata.obs[ctype_key]!='OB']

In [ ]:
# Separate species
adata_marmoset = adata[adata.obs['dataset']=='marmoset']
adata_mouse = adata[adata.obs['dataset']=='mouse']
adatas = [adata_marmoset, adata_mouse]

In [ ]:
# Filter cell types by amount in each dataset
ctypes = list(adata.obs[ctype_key].unique())
ctypes_filtered = []
for ctype in ctypes:
    marmoset_counts = adata_marmoset.obs[ctype_key].value_counts()[ctype]
    mouse_counts = adata_mouse.obs[ctype_key].value_counts()[ctype]
    if (marmoset_counts > 100) and (mouse_counts > 100):
        ctypes_filtered.append(ctype)

In [ ]:
# Set up variables
ctypes = adata.obs[ctype_key].unique()
ns = dict()
alphas = dict()
significances = dict()

# Set up p-values
reject_p = 0.01
corrected_reject_p = reject_p / len(ctypes) # Bonferroni correction

# Get statistics
for ctype in ctypes:
    mouse = adata_mouse[adata_mouse.obs[ctype_key]==ctype].obs['depth']
    marmoset = adata_marmoset[adata_marmoset.obs[ctype_key]==ctype].obs['depth']
    alpha = mannwhitneyu(mouse, marmoset).pvalue
    ns[ctype] = (len(mouse), len(marmoset))
    alphas[ctype] = alpha
    significances[ctype] = alpha < corrected_reject_p

In [ ]:
species_cmap = plt.cm.get_cmap('tab10').colors[:2][::-1]

In [ ]:
ctypes_filtered

In [ ]:
adata = adata[np.isin(adata.obs[ctype_key], ctypes_filtered)]

# Separate species
adata_marmoset = adata[adata.obs['dataset']=='marmoset']
adata_mouse = adata[adata.obs['dataset']=='mouse']
adatas = [adata_marmoset, adata_mouse]

In [ ]:
order = [
    'Astro', 'Endo', 'OPC', 'Oligo', 'Micro/Oligo', # Non-neuronal
    'IT', 'ET', # Excitatory
    'Vip', 'Cxcl14', 'Sst', 'Pvalb', # Inhibitory
]

In [ ]:
set(order) == set(adata.obs[ctype_key].unique())

In [ ]:
# Make main plot
plt.figure(figsize=(12,3))
sns.boxplot(
    data=adata.obs,
    x=ctype_key,
    y='depth',
    hue='dataset',
    palette=species_cmap[::-1],
    order=order
)
plt.gca().grid(False)
# plt.legend(bbox_to_anchor=(1,1))
plt.gca().get_legend().remove()

# Add significance indicators
max_depth = adata.obs['depth'].max()
buffer = 0.001
sns.despine()
ticks, labels = plt.xticks()
for i in range(len(ticks)):
    label = labels[i].get_text().split(',')[0]
    significant = significances[label]
    if significant:
        tick = ticks[i]
        plt.text(tick, max_depth+buffer, '*', ha='center')
        plt.plot([tick-0.3, tick+0.3], [max_depth+buffer, max_depth+buffer], color='k')

# Modify xticks
new_labels = []
for i in range(len(ticks)):
    label = labels[i].get_text().split(',')[0]
    new_labels.append(f'{label}, n={ns[label]}')
    # new_labels.append('')
plt.xticks(ticks, new_labels, rotation=90)

# Modify yticks
vmax = adata.obs['depth'].max()
vmin = adata.obs['depth'].min()
plt.yticks([vmax, vmin], ['', ''])
plt.gca().set_yticklabels('')
plt.xlabel('')
plt.ylabel('')

# plt.title('Ileum', fontsize=18)
figname = 'ctype_depth_boxplot.png'
savepath = os.path.join(basepath, figname)
# plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
# Make main plot
plt.figure(figsize=(12,3))
sns.boxplot(
    data=adata.obs,
    x=ctype_key,
    y='depth',
    hue='dataset',
    palette=species_cmap[::-1],
    order=order
)
plt.gca().grid(False)
# plt.legend(bbox_to_anchor=(1,1))
plt.gca().get_legend().remove()

# Add significance indicators
max_depth = adata.obs['depth'].max()
buffer = 0.001
sns.despine()
ticks, labels = plt.xticks()
for i in range(len(ticks)):
    label = labels[i].get_text().split(',')[0]
    significant = significances[label]
    if significant:
        tick = ticks[i]
        plt.text(tick, max_depth+buffer, '*', ha='center')
        plt.plot([tick-0.3, tick+0.3], [max_depth+buffer, max_depth+buffer], color='k')

# Modify xticks
new_labels = []
for i in range(len(ticks)):
    label = labels[i].get_text().split(',')[0]
    # new_labels.append(f'{label}, n={ns[label]}')
    new_labels.append('')
plt.xticks(ticks, new_labels, rotation=90)

# Modify yticks
vmax = adata.obs['depth'].max()
vmin = adata.obs['depth'].min()
plt.yticks([vmax, vmin], ['', ''])
plt.gca().set_yticklabels('')
plt.xlabel('')
plt.ylabel('')

# plt.title('Ileum', fontsize=18)
figname = 'ctype_depth_boxplot.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
def get_zoom(adata, x, y, width, height):
    zoomdata = adata[adata.obsm['spatial'][:,0]>x]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,0]<x+width]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,1]>y]
    zoomdata = zoomdata[zoomdata.obsm['spatial'][:,1]<y+height]
    return zoomdata

In [ ]:
# marmoset_box = (-4400, 10800, 1400, 4200)
marmoset_box = (-4400, 10200, 1600, 4800)
subdata = adata[adata.obs['dataset']=='marmoset']
plt.figure(figsize=(1.5,4))
plt.scatter(*subdata.obsm['spatial'].T, s=1)
rect = Rectangle(marmoset_box[:2], marmoset_box[2], marmoset_box[3], fill=False, edgecolor='k')
plt.gca().add_patch(rect)
plt.axis('off')
plt.show()

# mouse_box = (2600, 8200, 2050, 6150)
mouse_box = (2600, 8900, 1800, 5400)
subdata = adata[adata.obs['dataset']=='mouse']
plt.figure(figsize=(4,4))
plt.scatter(*subdata.obsm['spatial'].T, s=1)
rect = Rectangle(mouse_box[:2], mouse_box[2], mouse_box[3], fill=False, edgecolor='k')
plt.gca().add_patch(rect)
plt.axis('off')
plt.show()


zoomdata_marmoset = get_zoom(adata[adata.obs['dataset']=='marmoset'], *marmoset_box)
zoomdata_mouse = get_zoom(adata[adata.obs['dataset']=='mouse'], *mouse_box)

In [ ]:
ctype = 'Pvalb'
sc.set_figure_params(figsize=(2,6))
species = ['marmoset', 'mouse']
zoomdatas = [zoomdata_marmoset, zoomdata_mouse]
for i in range(len(species)):
    sc.pl.embedding(zoomdatas[i], basis='spatial', color=ctype_key, groups=ctype, palette=[species_cmap[::-1][i]], s=100, return_fig=True, frameon=False, title='', legend_loc=None, edgecolor=None)
    # sc.pl.embedding(zoomdatas[i], basis='spatial', color=ctype_key, groups=ctype, palette='k', s=100, return_fig=True, frameon=False, title='', legend_loc=None, edgecolor=None)
    figname = f'zoom_{species[i]}_colored_by_{ctype}.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()

## Plot hist for physical depth in each slice

In [ ]:
# Isolate a given cell type
zoomdata_marmoset_ctype = zoomdata_marmoset[zoomdata_marmoset.obs[ctype_key]==ctype]
zoomdata_mouse_ctype = zoomdata_mouse[zoomdata_mouse.obs[ctype_key]==ctype]

In [ ]:
physdepth_marmoset = zoomdata_marmoset_ctype.obsm['spatial'][:,1].copy()
physdepth_marmoset -= physdepth_marmoset.min()
physdepth_marmoset /= physdepth_marmoset.ptp()
plt.figure(figsize=(10,1))
plt.hist(physdepth_marmoset, color=species_cmap[1])
plt.axis('off')
figname = f'hist_for_zoom_marmoset_{ctype}.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
physdepth_mouse = zoomdata_mouse_ctype.obsm['spatial'][:,1].copy()
physdepth_mouse -= physdepth_mouse.min()
physdepth_mouse /= physdepth_mouse.ptp()
plt.figure(figsize=(10,1))
plt.hist(physdepth_mouse, color=species_cmap[0])
plt.axis('off')
figname = f'hist_for_zoom_mouse_{ctype}.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

## Higher res cell types

In [ ]:
# Set up variables
ctype_key = 'celltype'
ctypes = adata.obs[ctype_key].unique()
ns = dict()
alphas = dict()
significances = dict()

# Set up p-values
reject_p = 0.01
corrected_reject_p = 0.05 / len(ctypes) # Bonferroni correction

# Get statistics
for ctype in ctypes:
    mouse = adata_mouse[adata_mouse.obs[ctype_key]==ctype].obs['depth']
    marmoset = adata_marmoset[adata_marmoset.obs[ctype_key]==ctype].obs['depth']
    alpha = mannwhitneyu(mouse, marmoset).pvalue
    ns[ctype] = (len(mouse), len(marmoset))
    alphas[ctype] = alpha
    significances[ctype] = alpha < corrected_reject_p

In [ ]:
# Make main plot
plt.figure(figsize=(12,3))
sns.boxplot(
    data=adata.obs,
    x=ctype_key,
    y='depth',
    hue='dataset',
    palette=species_cmap[::-1]
)
plt.gca().grid(False)
# plt.legend(bbox_to_anchor=(1,1))
plt.gca().get_legend().remove()

# Add significance indicators
max_depth = adata.obs['depth'].max()
buffer = 0.05
sns.despine()
ticks, labels = plt.xticks()
for i in range(len(ticks)):
    label = labels[i].get_text().split(',')[0]
    significant = significances[label]
    if significant:
        tick = ticks[i]
        plt.text(tick, max_depth+buffer, '*', ha='center')
        plt.plot([tick-0.3, tick+0.3], [max_depth+buffer, max_depth+buffer], color='k')

# Modify xticks
new_labels = []
for i in range(len(ticks)):
    label = labels[i].get_text().split(',')[0]
    # new_labels.append(f'{label}, n={ns[label]}')
    new_labels.append('')
plt.xticks(ticks, new_labels, rotation=90)

# Modify yticks
vmax = adata.obs['depth'].max()
vmin = adata.obs['depth'].min()
plt.yticks([vmax, vmin], ['', ''])
plt.gca().set_yticklabels('')
plt.xlabel('')
plt.ylabel('')

# plt.title('Ileum', fontsize=18)
figname = 'ctype_depth_boxplot.png'
savepath = os.path.join(basepath, figname)
plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
plt.show()

In [ ]:
ctype = 'Sncg'
sc.set_figure_params(figsize=(2,6))
species = ['marmoset', 'mouse']
zoomdatas = [zoomdata_marmoset, zoomdata_mouse]
for i in range(len(species)):
    sc.pl.embedding(zoomdatas[i], basis='spatial', color=ctype_key, groups=ctype, palette=['k'], s=100, return_fig=True, frameon=False, title='', legend_loc=None, edgecolor=None)
    figname = f'zoom_{species[i]}_colored_by_Sncg.png'
    savepath = os.path.join(basepath, figname)
    plt.savefig(savepath, bbox_inches='tight', transparent=True, dpi=dpi)
    plt.show()